In [1]:
import pandas as pd
import afqinsight.nn.tf_models as nn
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from afqinsight.datasets import AFQDataset
from afqinsight.nn.tf_models import cnn_lenet, mlp4, cnn_vgg, lstm1v0, lstm1, lstm2, blstm1, blstm2, lstm_fcn, cnn_resnet
from sklearn.impute import SimpleImputer
import os.path
# Harmonization
from sklearn.model_selection import train_test_split
from neurocombat_sklearn import CombatModel
from sklearn.metrics import r2_score, median_absolute_error, mean_absolute_error

import pickle
from tools import load_data, model_fit

In [2]:
X, y, site = load_data()

2022-07-13 02:10:59.624421: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
from sklearn.utils import shuffle, resample

In [4]:
from sklearn.metrics import r2_score, median_absolute_error

In [5]:
model_dict = {
  "cnn_lenet": {"model": cnn_lenet, "lr": 0.001}, 
  # "mlp4": {"model": mlp4, "lr": 0.001},
  # "cnn_vgg": {"model": cnn_vgg, "lr": 0.001},
  # "lstm1v0": {"model": lstm1v0, "lr": 0.01},
  # "lstm1": {"model": lstm1, "lr": 0.01},
  # "lstm2": {"model": lstm2, "lr": 0.01},
  # "blstm1": {"model": blstm1, "lr": 0.01},
  # "blstm2": {"model": blstm1, "lr": 0.01},
  # "lstm_fcn": {"model": lstm_fcn, "lr": 0.01},
#   "cnn_resnet": {"model": cnn_resnet, "lr": 0.01}
             }

In [6]:
n_runs = 10

In [7]:
# Generated once with this code and then hard-coded:
# seeds = np.array([np.abs(np.floor(np.random.randn()*1000)) for ii in range(n_runs)], dtype=int)

seeds = np.array([484, 645, 714, 244, 215, 1502, 1334, 1576, 469, 1795])

In [8]:
def fit_and_eval(model, X, y, random_state, batch_size=128, n_epochs=1000, augment=None, train_size=None):
    model_func = model_dict[model]["model"]
    lr = model_dict[model]["lr"]
    X_train, X_test, y_train, y_test, site_train, site_test = train_test_split(X, y, site, test_size=0.2, random_state=random_state)
    imputer = SimpleImputer(strategy="median")
    # If train_size is set, select train_size subjects to be the training data:
    if train_size is not None:
        X_train, y_train, site_train = resample(X_train, y_train, site_train, replace=False, n_samples=train_size, random_state=random_state)
    
    # Impute train and test separately:
    X_train = np.concatenate([imputer.fit_transform(X_train[..., ii])[:, :, None] for ii in range(X_train.shape[-1])], -1)
    X_test = np.concatenate([imputer.fit_transform(X_test[..., ii])[:, :, None] for ii in range(X_test.shape[-1])], -1)
    # Combat
    X_train = np.concatenate([CombatModel().fit_transform(X_train[..., ii], site_train[:, None], None, None)[:, :, None] for ii in range(X_train.shape[-1])], -1)
    X_test = np.concatenate([CombatModel().fit_transform(X_test[..., ii], site_test[:, None], None, None)[:, :, None] for ii in range(X_test.shape[-1])], -1)
    
    trained = model_fit(model_func, X_train, y_train, lr, 
                        batch_size=batch_size, n_epochs=n_epochs)
    metric = []
    value = []
    
    y_pred = trained.predict(X_test)
    metric.append("mae")
    value.append(mean_absolute_error(y_test, y_pred))
    metric.append("mad")
    value.append(median_absolute_error(y_test, y_pred))
    metric.append("r2")
    value.append(r2_score(y_test, y_pred))
    
    result = {'Model': [model] * len(metric),
              'Metric': metric,
              'Value': value}
    
    return pd.DataFrame(result), pd.DataFrame(dict(y_pred=y_pred.squeeze(), y_test=y_test))

In [9]:
dfs_eval = []
dfs_pred = []

In [10]:
for model in model_dict:
    print("##################################################")
    print("model: ", model)
    for ii in range(n_runs): 
        print("run: ", ii)
        this_eval, this_pred = fit_and_eval(
            model, 
            X, 
            y, 
            random_state=seeds[ii],
            train_size=700)
        this_eval["run"] = ii
        this_pred["run"] = ii
        dfs_eval.append(this_eval)
        dfs_pred.append(this_pred)
        # Save evaluation metrics
        one_df = pd.concat(dfs_eval)
        one_df.to_csv("lenet_3_eval.csv")
        # Save predictions and test values:
        one_df = pd.concat(dfs_pred)
        one_df.to_csv("lenet_3_pred.csv")

##################################################
model:  cnn_lenet
run:  0
pooling layers: 4
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 100, 72)]         0         
                                                                 
 conv1d (Conv1D)             (None, 100, 6)            1302      
                                                                 
 max_pooling1d (MaxPooling1D  (None, 50, 6)            0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 50, 16)            304       
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 25, 16)           0         
 1D)                                                             
                                

Epoch 21/1000
5/5 [==============================] - 0s 16ms/step - loss: 16.9803 - mean_squared_error: 16.9803 - rmse: 4.1207 - mean_absolute_error: 3.2224 - val_loss: 8.3486 - val_mean_squared_error: 8.3486 - val_rmse: 2.8894 - val_mean_absolute_error: 2.2327 - lr: 0.0010
Epoch 22/1000
5/5 [==============================] - 0s 19ms/step - loss: 16.2391 - mean_squared_error: 16.2391 - rmse: 4.0298 - mean_absolute_error: 3.1638 - val_loss: 7.9652 - val_mean_squared_error: 7.9652 - val_rmse: 2.8223 - val_mean_absolute_error: 2.2008 - lr: 0.0010
Epoch 23/1000
5/5 [==============================] - 0s 16ms/step - loss: 16.0184 - mean_squared_error: 16.0184 - rmse: 4.0023 - mean_absolute_error: 3.0361 - val_loss: 8.0902 - val_mean_squared_error: 8.0902 - val_rmse: 2.8443 - val_mean_absolute_error: 2.2041 - lr: 0.0010
Epoch 24/1000
5/5 [==============================] - 0s 17ms/step - loss: 16.0121 - mean_squared_error: 16.0121 - rmse: 4.0015 - mean_absolute_error: 3.1779 - val_loss: 8.8250

Epoch 51/1000
5/5 [==============================] - 0s 18ms/step - loss: 14.3802 - mean_squared_error: 14.3802 - rmse: 3.7921 - mean_absolute_error: 2.9613 - val_loss: 7.1960 - val_mean_squared_error: 7.1960 - val_rmse: 2.6825 - val_mean_absolute_error: 2.2266 - lr: 0.0010
Epoch 52/1000
5/5 [==============================] - 0s 18ms/step - loss: 14.4473 - mean_squared_error: 14.4473 - rmse: 3.8010 - mean_absolute_error: 2.9893 - val_loss: 8.0808 - val_mean_squared_error: 8.0808 - val_rmse: 2.8427 - val_mean_absolute_error: 2.2197 - lr: 0.0010
Epoch 53/1000
5/5 [==============================] - 0s 17ms/step - loss: 14.0278 - mean_squared_error: 14.0278 - rmse: 3.7454 - mean_absolute_error: 2.9507 - val_loss: 7.1901 - val_mean_squared_error: 7.1901 - val_rmse: 2.6814 - val_mean_absolute_error: 2.1596 - lr: 0.0010
Epoch 54/1000
5/5 [==============================] - 0s 16ms/step - loss: 15.1866 - mean_squared_error: 15.1866 - rmse: 3.8970 - mean_absolute_error: 2.9931 - val_loss: 7.0854

Epoch 81/1000
5/5 [==============================] - ETA: 0s - loss: 13.7910 - mean_squared_error: 13.7910 - rmse: 3.7136 - mean_absolute_error: 2.9087
Epoch 00081: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
5/5 [==============================] - 0s 16ms/step - loss: 13.7910 - mean_squared_error: 13.7910 - rmse: 3.7136 - mean_absolute_error: 2.9087 - val_loss: 6.9065 - val_mean_squared_error: 6.9065 - val_rmse: 2.6280 - val_mean_absolute_error: 2.1661 - lr: 0.0010
Epoch 82/1000
5/5 [==============================] - 0s 16ms/step - loss: 12.3123 - mean_squared_error: 12.3123 - rmse: 3.5089 - mean_absolute_error: 2.7575 - val_loss: 6.9090 - val_mean_squared_error: 6.9090 - val_rmse: 2.6285 - val_mean_absolute_error: 2.1585 - lr: 5.0000e-04
Epoch 83/1000
5/5 [==============================] - 0s 15ms/step - loss: 11.7943 - mean_squared_error: 11.7943 - rmse: 3.4343 - mean_absolute_error: 2.6807 - val_loss: 6.9568 - val_mean_squared_error: 6.9568 - val_rmse: 2.6376 

Epoch 110/1000
5/5 [==============================] - 0s 15ms/step - loss: 10.3139 - mean_squared_error: 10.3139 - rmse: 3.2115 - mean_absolute_error: 2.5427 - val_loss: 7.6898 - val_mean_squared_error: 7.6898 - val_rmse: 2.7731 - val_mean_absolute_error: 2.2786 - lr: 5.0000e-04
Epoch 111/1000
5/5 [==============================] - 0s 16ms/step - loss: 11.9090 - mean_squared_error: 11.9090 - rmse: 3.4509 - mean_absolute_error: 2.6878 - val_loss: 7.0765 - val_mean_squared_error: 7.0765 - val_rmse: 2.6602 - val_mean_absolute_error: 2.1673 - lr: 5.0000e-04
Epoch 112/1000
5/5 [==============================] - 0s 16ms/step - loss: 10.8346 - mean_squared_error: 10.8346 - rmse: 3.2916 - mean_absolute_error: 2.5689 - val_loss: 7.4108 - val_mean_squared_error: 7.4108 - val_rmse: 2.7223 - val_mean_absolute_error: 2.2251 - lr: 5.0000e-04
Epoch 113/1000
5/5 [==============================] - 0s 15ms/step - loss: 10.6998 - mean_squared_error: 10.6998 - rmse: 3.2711 - mean_absolute_error: 2.5529 - 

...

Epoch 221/1000
5/5 [==============================] - 0s 19ms/step - loss: 7.2075 - mean_squared_error: 7.2075 - rmse: 2.6847 - mean_absolute_error: 2.0705 - val_loss: 6.2388 - val_mean_squared_error: 6.2388 - val_rmse: 2.4978 - val_mean_absolute_error: 1.8934 - lr: 2.5000e-04
Epoch 222/1000
5/5 [==============================] - 0s 19ms/step - loss: 7.7328 - mean_squared_error: 7.7328 - rmse: 2.7808 - mean_absolute_error: 2.1098 - val_loss: 5.9222 - val_mean_squared_error: 5.9222 - val_rmse: 2.4336 - val_mean_absolute_error: 1.9122 - lr: 2.5000e-04
Epoch 223/1000
5/5 [==============================] - 0s 19ms/step - loss: 7.8972 - mean_squared_error: 7.8972 - rmse: 2.8102 - mean_absolute_error: 2.1684 - val_loss: 6.0916 - val_mean_squared_error: 6.0916 - val_rmse: 2.4681 - val_mean_absolute_error: 1.8789 - lr: 2.5000e-04
Epoch 224/1000
5/5 [==============================] - 0s 19ms/step - loss: 8.0031 - mean_squared_error: 8.0031 - rmse: 2.8290 - mean_absolute_error: 2.1614 - val_loss

Epoch 250/1000
5/5 [==============================] - 0s 18ms/step - loss: 7.2765 - mean_squared_error: 7.2765 - rmse: 2.6975 - mean_absolute_error: 2.0557 - val_loss: 5.8213 - val_mean_squared_error: 5.8213 - val_rmse: 2.4127 - val_mean_absolute_error: 1.8515 - lr: 1.2500e-04
Epoch 251/1000
5/5 [==============================] - 0s 18ms/step - loss: 8.2161 - mean_squared_error: 8.2161 - rmse: 2.8664 - mean_absolute_error: 2.1188 - val_loss: 5.8467 - val_mean_squared_error: 5.8467 - val_rmse: 2.4180 - val_mean_absolute_error: 1.8467 - lr: 1.2500e-04
Epoch 252/1000
5/5 [==============================] - 0s 17ms/step - loss: 7.5328 - mean_squared_error: 7.5328 - rmse: 2.7446 - mean_absolute_error: 2.1581 - val_loss: 5.7351 - val_mean_squared_error: 5.7351 - val_rmse: 2.3948 - val_mean_absolute_error: 1.8509 - lr: 1.2500e-04
Epoch 253/1000
5/5 [==============================] - 0s 18ms/step - loss: 7.8910 - mean_squared_error: 7.8910 - rmse: 2.8091 - mean_absolute_error: 2.1548 - val_loss

Epoch 279/1000
5/5 [==============================] - 0s 20ms/step - loss: 7.4644 - mean_squared_error: 7.4644 - rmse: 2.7321 - mean_absolute_error: 2.0927 - val_loss: 5.8692 - val_mean_squared_error: 5.8692 - val_rmse: 2.4227 - val_mean_absolute_error: 1.8315 - lr: 6.2500e-05
Epoch 280/1000
5/5 [==============================] - 0s 20ms/step - loss: 7.1435 - mean_squared_error: 7.1435 - rmse: 2.6727 - mean_absolute_error: 2.0607 - val_loss: 5.7387 - val_mean_squared_error: 5.7387 - val_rmse: 2.3956 - val_mean_absolute_error: 1.8304 - lr: 6.2500e-05
Epoch 281/1000
5/5 [==============================] - 0s 22ms/step - loss: 7.2627 - mean_squared_error: 7.2627 - rmse: 2.6949 - mean_absolute_error: 2.0618 - val_loss: 5.7068 - val_mean_squared_error: 5.7068 - val_rmse: 2.3889 - val_mean_absolute_error: 1.8338 - lr: 6.2500e-05
Epoch 282/1000
5/5 [==============================] - 0s 31ms/step - loss: 7.6727 - mean_squared_error: 7.6727 - rmse: 2.7700 - mean_absolute_error: 2.0758 - val_loss

Epoch 308/1000
5/5 [==============================] - 0s 19ms/step - loss: 7.3835 - mean_squared_error: 7.3835 - rmse: 2.7173 - mean_absolute_error: 2.0973 - val_loss: 5.7095 - val_mean_squared_error: 5.7095 - val_rmse: 2.3894 - val_mean_absolute_error: 1.8416 - lr: 3.1250e-05
Epoch 309/1000
5/5 [==============================] - 0s 19ms/step - loss: 7.6878 - mean_squared_error: 7.6878 - rmse: 2.7727 - mean_absolute_error: 2.0882 - val_loss: 5.7387 - val_mean_squared_error: 5.7387 - val_rmse: 2.3956 - val_mean_absolute_error: 1.8371 - lr: 3.1250e-05
Epoch 310/1000
5/5 [==============================] - 0s 19ms/step - loss: 6.8869 - mean_squared_error: 6.8869 - rmse: 2.6243 - mean_absolute_error: 1.9886 - val_loss: 5.7462 - val_mean_squared_error: 5.7462 - val_rmse: 2.3971 - val_mean_absolute_error: 1.8369 - lr: 3.1250e-05
Epoch 311/1000
5/5 [==============================] - 0s 18ms/step - loss: 7.4881 - mean_squared_error: 7.4881 - rmse: 2.7364 - mean_absolute_error: 2.0876 - val_loss

Epoch 337/1000
5/5 [==============================] - 0s 18ms/step - loss: 7.2464 - mean_squared_error: 7.2464 - rmse: 2.6919 - mean_absolute_error: 2.0540 - val_loss: 5.7388 - val_mean_squared_error: 5.7388 - val_rmse: 2.3956 - val_mean_absolute_error: 1.8322 - lr: 7.8125e-06
Epoch 338/1000
5/5 [==============================] - 0s 18ms/step - loss: 7.0081 - mean_squared_error: 7.0081 - rmse: 2.6473 - mean_absolute_error: 2.0424 - val_loss: 5.7581 - val_mean_squared_error: 5.7581 - val_rmse: 2.3996 - val_mean_absolute_error: 1.8302 - lr: 7.8125e-06
Epoch 339/1000
5/5 [==============================] - 0s 18ms/step - loss: 7.6669 - mean_squared_error: 7.6669 - rmse: 2.7689 - mean_absolute_error: 2.1201 - val_loss: 5.7791 - val_mean_squared_error: 5.7791 - val_rmse: 2.4040 - val_mean_absolute_error: 1.8297 - lr: 7.8125e-06
Epoch 340/1000
5/5 [==============================] - 0s 18ms/step - loss: 7.3802 - mean_squared_error: 7.3802 - rmse: 2.7167 - mean_absolute_error: 2.0845 - val_loss

In [11]:
# We want to try different training sample sizes: s
# (1453, 1000, 700, 350, 175)